In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import yaml
from PIL import Image

: 

In [ ]:
dataset_path = r"/home/bhavesh/pathpal/datasets/grocery/dataset2"
train_path = os.path.join(dataset_path,'train')
val_path = os.path.join(dataset_path,'valid')
test_path = os.path.join(dataset_path,'test')
train_images_path = os.path.join(dataset_path,'train/images')
train_labels_path = os.path.join(dataset_path,'train/labels')
val_images_path = os.path.join(dataset_path,'valid/labels')
val_labels_path = os.path.join(dataset_path,'valid/labels')
test_images_path = os.path.join(dataset_path,'test/images')
test_labels_path = os.path.join(dataset_path,'test/labels')

: 

In [ ]:
os.path.exists(dataset_path)

: 

In [ ]:
with open(os.path.join(dataset_path, 'data.yaml')) as file:
    yaml_data = yaml.load(file, Loader=yaml.FullLoader)
classes = yaml_data['names']
print(classes)
print(len(classes))

: 

In [ ]:
def load_data(images_path, labels_path):
    image_files = [f for f in os.listdir(images_path) if f.endswith(('.jpg', '.png','.jpeg'))]
    label_files_unordered = [f for f in os.listdir(labels_path) if f.endswith('.txt')]
    
    label_files = []

    for image_file in image_files:
        label_file = os.path.join(labels_path,image_file.replace(os.path.splitext(image_file)[1], '.txt') )
        if os.path.exists(label_file):
            label_files.append(label_file)
        else:
            print(label_file)
            print('what image is this bro?')
            pass
    
    images = []
    labels = []

    for image_file, label_file in zip(image_files, label_files):
        image_path = os.path.join(images_path, image_file)
        image = Image.open(image_path)
        images.append(image)

        label_path = label_file
        with open(label_path, 'r') as f:
            lines = f.readlines()
            
        boxes = []
        for line in lines:
            parts = line.strip().split()
            class_index = int(parts[0])  
            x_center = float(parts[1])
            y_center = float(parts[2])
            width = float(parts[3])
            height = float(parts[4])

            boxes.append({
                'class': class_index,
                'x_center': x_center,
                'y_center': y_center,
                'width': width,
                'height': height
            })
        labels.append(boxes)
    
    return images, labels    

: 

In [ ]:
images, labels = load_data(train_images_path, train_labels_path )

: 

In [ ]:
def display_sample(i):
    img = images[i]
    label = labels[i]

    fig, ax = plt.subplots(1, figsize=(5, 5))
    
    # Display the image on the axis
    ax.imshow(img)
    
    # Loop through each bounding box in the label data and draw it
    for box in label:
        # Get bounding box coordinates (in relative format)
        x_center = box['x_center']
        y_center = box['y_center']
        width = box['width']
        height = box['height']
        
        # Convert from relative to absolute pixel coordinates
        image_width, image_height = img.size
        x_min = (x_center - width / 2) * image_width
        y_min = (y_center - height / 2) * image_height
        x_max = (x_center + width / 2) * image_width
        y_max = (y_center + height / 2) * image_height
        
        # Draw the bounding box on the image using matplotlib's Rectangle
        ax.add_patch(plt.Rectangle(
            (x_min, y_min), x_max - x_min, y_max - y_min, 
            linewidth=2, edgecolor='red', facecolor='none'
        ))
        ax.text(
            x_min, y_min - 10,  # Position above the bounding box
            classes[box['class']],  # Class name
            color='red', fontsize=12, fontweight='bold'
            # bbox=dict(facecolor='white', alpha=0.5, edgecolor='red', boxstyle='round,pad=0.5')
        )

    # Remove axis and display the image with bounding boxes
    ax.axis('off')
    plt.show()

: 

In [ ]:
import random
i = random.randint(0,len(images))
display_sample(i)

: 